Задача — создать Python-функцию, которая будет проверять входящие сообщения и выявлять скрытые в них ссылки для дальнейшего анализа.

Основные компоненты регулярных выражений:

- Шаблоны: По сути, это набор правил, которым следует regex при поиске данных.
- Метасимволы: Это специальные символы, которые имеют уникальное значение для модуля regex. К ним относятся такие символы, как {}[]()^$.*?|\.
- Квантификаторы: Они определяют, сколько раз должен повторяться тот или иной участок шаблона. К общим квантификаторам относятся * (0 или более раз), + (1 или более раз) и ? (0 или 1 раз).
- Специальные последовательности: Это последовательности символов, которые имеют специальное значение, например, \d означает любую цифру, или \s любой пробельный символ.

Борьба с вредоносными URL-адресами
Спамеры часто пытаются скрыть свои намерения, и один из распространённых способов — маскировка URL-адресов.

Вот несколько популярных приемов:

Сокращатели URL-адресов: Спамеры часто используют такие сервисы, как Bitly, TinyURL, чтобы сократить свои URL-адреса и сделать их менее узнаваемыми.
Использование Punycode: Punycode позволяет кодировать Unicode символы в ASCII, что часто используется спамерами для маскировки URL. Например, "www.xn--80ak6aa92e.com" на самом деле является "www.аррӏе.com", что может ввести в заблуждение пользователей, думающих, что они посещают "www.apple.com".
Скрытие на виду: Некоторые спамеры вставляют URL прямо в текст без использования стандартного формата (например, "example.com" вместо "http://www.example.com").

### Задание
Ваша задача — написать функцию parse_urls. Эта функция должна принимать строку сообщения, обнаруживать в ней URL-адреса с помощью регулярных выражений, проверять доступность этих URL и, в конечном итоге, возвращать словарь. В этом словаре ключи — это уникальные URL (нормализованные так, чтобы в них не входили "www", "http://" или "https://"), а значения — это количество их упоминаний в строке сообщения.

Доступность URL в данной задаче нужно проверить с помощью библиотеки requests таким образом:

In [13]:
import re
from collections import Counter
from typing import Dict, List
import requests
def normalize_domains(domains):
    normalized_domains = []
    for domain in domains:
        # Убираем пробелы, точки в конце и слэш, если он есть
        clean_domain = domain.strip().rstrip('/').rstrip('.')
        normalized_domains.append(clean_domain)
    return normalized_domains
        

def parse_urls(message: str) -> Dict[str, int]:
    """Parse a list of URL strings and check their reachability."""
    # pattern = re.compile(r'(https?://)?(www\.)?([a-zA-Z0-9-]+\.(com|org|co\.uk)(/[\w\-./?%&=]*)?)')
    # pattern = re.compile(r'(https?://)?(www\.)?([a-zA-Z0-9-]+\.(com|org|co\.uk))(/[\w\-./?%&=]*)?')
    #?%&=
    pattern = re.compile(r'(https?://)?(www\.)?([a-zA-Z0-9.-]+\.(com|org|co\.uk)(/[\w\-./]*)?)')


    matches = pattern.findall(message)
    domains = [match[2] for match in matches]
    new_domains = normalize_domains(domains)
    

    # Проверка доступности URL
    reachable_urls = check_url_reachability(new_domains)

    # Подсчет только доступных доменов
    domain_count = Counter(reachable_urls)

    return dict(domain_count)


def check_url_reachability(urls: List[str]) -> List[str]:
    """Check the reachability of the list of URLs."""
    reachable_urls = []
    for url in urls:
        # Добавляем https:// перед проверкой
        full_url = f"https://{url}"

        try:
            response = requests.head(full_url, allow_redirects=True, timeout=5)
            if response.status_code == 200:
                reachable_urls.append(url)  # Сохраняем домен без протокола
                continue
        except requests.RequestException:
            pass

        try:
            response = requests.head(full_url, allow_redirects=False, timeout=5)
            if response.status_code == 200:
                reachable_urls.append(url)  # Сохраняем домен без протокола
        except requests.RequestException as e:
            print(e)

    return reachable_urls

if __name__ == "__main__":
    message = (
        "Quick visit to https://www.wikipedia.org/ for a wealth of knowledge."
        "Searching for recipes? Check http://allrecipes.com for inspiration."
        "For tech enthusiasts, https://www.wired.com is the place to be."
        "Connect with professionals at https://www.linkedin.com and expand your network."
        "Find your dream job at https://www.indeed.com or https://www.monster.com."
        "https://stackoverflow.com/questions/tagged/python has answers to many Python questions."
        "Discover amazing places on https://www.tripadvisor.com or book flights at https://www.kayak.com."
        "Enjoy shopping at https://www.amazon.com? Find similar deals at https://www.ebay.com."
        "Stay updated with news on https://www.bbc.co.uk or https://www.cnn.com."
        "Long string with multiple URLs: Check out the news at https://www.nytimes.com, grab some tech gadgets from https://www.bestbuy.com, find home improvement supplies at https://www.homedepot.com, look for health information on https://www.webmd.com, learn coding at https://www.codecademy.com, get financial advice from https://www.forbes.com, and explore https://en.wikipedia.org/wiki/Special:Random for a random Wikipedia article."
    )
    print(parse_urls(message))


HTTPSConnectionPool(host='monster.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001874922C410>: Failed to resolve 'monster.com' ([Errno 11002] getaddrinfo failed)"))
{'wikipedia.org': 1, 'wired.com': 1, 'linkedin.com': 1, 'stackoverflow.com/questions/tagged/python': 1, 'kayak.com': 1, 'bbc.co.uk': 1, 'cnn.com': 1, 'nytimes.com': 1, 'codecademy.com': 1, 'forbes.com': 1, 'en.wikipedia.org/wiki/Special': 1}


In [ ]:
import re
from collections import Counter
from typing import Dict

import requests


def add_http_scheme(url: str) -> str:
    if not url.startswith(("http://", "https://")):
        return "http://" + url
    return url


def parse_urls(message: str) -> Dict[str, int]:
    # Improved regex pattern to detect URLs without 'www' or 'http'
    pattern = r'https?://[^\s<>",]+|www\.[^\s<>",]+|(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}'

    # Find all matches in the message
    url_list = re.findall(pattern, message)

    # Add http scheme if missing, and check if they are reachable
    reachable_urls = []
    print(url_list)
    for url in url_list:
        url_with_scheme = add_http_scheme(url)
        try:
            response = requests.head(
                url_with_scheme,
                timeout=5,
            )
            print(url_with_scheme, response.status_code)
            url_norm = url_with_scheme.replace("http://", "")
            url_norm = url_norm.replace("https://", "")
            url_norm = url_norm.replace("www.", "")

            # remove punctuation at the end of url
            if url_norm.endswith((".", ",", "!", "?")):
                url_norm = url_norm[:-1]

            # remove / at the end of url
            if url_norm.endswith("/"):
                url_norm = url_norm[:-1]

            reachable_urls.append(url_norm)
        except requests.RequestException as e:
            # Handle exceptions for requests
            print(e)

    # Create a Counter object to count unique URLs
    url_counts = Counter(reachable_urls)
    unique_url_dict = dict(url_counts)
    return unique_url_dict


if __name__ == "__main__":
    message = (
        "Check out this link www.example.com, example.com and"
        " also https://www.xn--80ak6aa92e.com/"
        " also www.xn--80ak6aa92e.com"
        " also xn--80ak6aa92e.com"
        " also apple.com"
        " Don't miss this great opportunity!"
        " www.google.com."
        " hello.ru"
    )
    print(parse_urls(message))
